In [1]:
# %reset 
# this mounts your Google Drive to the Colab VM.
from google.colab import drive
drive.mount('/content/drive', force_remount=True)

# enter the foldername in your Drive where you have saved the unzipped
# assignment folder, e.g. 'cs231n/assignments/assignment3/'
FOLDERNAME = 'Colab\ Notebooks/cs231n/project/SelfDeblur-master'
assert FOLDERNAME is not None, "[!] Enter the foldername."
repo_path = '/content/drive/My Drive/{}'.format(FOLDERNAME.replace('\\', ''))

# now that we've mounted your Drive, this ensures that
# the Python interpreter of the Colab VM can load
# python files from within it.
import sys
sys.path.append(repo_path)

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


In [0]:
from __future__ import print_function
import matplotlib.pyplot as plt
import argparse
import os
import numpy as np
from networks.skip import skip
from networks.fcn import *
import cv2
import torch
import torch.optim
import glob
from skimage.io import imread
from skimage.io import imsave
import warnings
from tqdm import tqdm
from torch.optim.lr_scheduler import MultiStepLR
from utils.common_utils import *
from SSIM import SSIM

In [0]:
torch.backends.cudnn.enabled = True
torch.backends.cudnn.benchmark =True
dtype = torch.cuda.FloatTensor

warnings.filterwarnings("ignore")

In [0]:
from recordtype import recordtype
Args = recordtype("Arguments", "num_iter img_size kernel_size data_path save_path save_frequency")
opt = Args(num_iter=1000,
           img_size=[256, 256], 
           kernel_size=[51, 51], 
           data_path="datasets/real/",
           save_path="results/real",
           save_frequency=100)

In [0]:
# files_source = glob.glob(os.path.join(repo_path, opt.data_path, '*.jpg'))
# files_source.sort()
files_source = os.path.join(repo_path, opt.data_path, 'jack.jpg')
os.makedirs(os.path.join(repo_path, opt.save_path), exist_ok=True)
files_source = [files_source]

In [0]:
# start #image
# del net_input, net_input_kernel
for f in files_source:
    INPUT = 'noise'
    pad = 'reflection'
    LR = 0.01
    num_iter = opt.num_iter
    reg_noise_std = 0.001

    path_to_image = f
    imgname = os.path.basename(f)
    imgname = os.path.splitext(imgname)[0]

    if imgname.find('fish') != -1:
        opt.kernel_size = [41, 41]
    if imgname.find('flower') != -1:
        opt.kernel_size = [25, 25]
    if imgname.find('house') != -1:
        opt.kernel_size = [51, 51]
    
    print(path_to_image)
    img, y, cb, cr = readimg(path_to_image)
    y = np.float32(y / 255.0)
    y = np.expand_dims(y, 0)
    img_size = y.shape

    print(imgname)
    # ######################################################################

    padw, padh = opt.kernel_size[0]-1, opt.kernel_size[1]-1
    opt.img_size[0], opt.img_size[1] = img_size[1]+padw, img_size[2]+padh
    #y = y[:, padh//2:img_size[1]-padh//2, padw//2:img_size[2]-padw//2]
    y = np_to_torch(y).type(dtype)

    input_depth = 8

    net_input = get_noise(input_depth, INPUT, (opt.img_size[0], opt.img_size[1])).type(dtype).detach()

    net = skip(input_depth, 1,
               num_channels_down=[128, 128, 128, 128, 128],
               num_channels_up=[128, 128, 128, 128, 128],
               num_channels_skip=[16, 16, 16, 16, 16],
               upsample_mode='bilinear',
               need_sigmoid=True, need_bias=True, pad=pad, act_fun='LeakyReLU')

    net = net.type(dtype)

    n_k = 200
    net_input_kernel = get_noise(n_k, INPUT, (1, 1)).type(dtype)
    with torch.no_grad():
      net_input_kernel.squeeze_()

    net_kernel = fcn(n_k, opt.kernel_size[0] * opt.kernel_size[1])
    net_kernel = net_kernel.type(dtype)

    # Losses
    mse = torch.nn.MSELoss().type(dtype)
    ssim = SSIM().type(dtype)

    # optimizer
    optimizer = torch.optim.Adam([{'params': net.parameters()}, {'params': net_kernel.parameters(), 'lr': 1e-4}], lr=LR)
    scheduler = MultiStepLR(optimizer, milestones=[1600, 1900, 2200], gamma=0.5)  # learning rates

    # initilization inputs
    net_input_saved = net_input.detach().clone()
    net_input_kernel_saved = net_input_kernel.detach().clone()

    ### start SelfDeblur
    for step in tqdm(range(num_iter)):

        # input regularization
        net_input = net_input_saved + reg_noise_std * torch.zeros(net_input_saved.shape).type_as(
            net_input_saved.data).normal_()
        # net_input_kernel = net_input_kernel_saved + reg_noise_std*torch.zeros(net_input_kernel_saved.shape).type_as(net_input_kernel_saved.data).normal_()

        # change the learning rate
        scheduler.step(step)
        optimizer.zero_grad()

        # get the network output
        out_x = net(net_input)
        out_k = net_kernel(net_input_kernel)

        out_k_m = out_k.view(-1, 1, opt.kernel_size[0], opt.kernel_size[1])

        # print(out_k_m)
        out_y = nn.functional.conv2d(out_x, out_k_m, padding=0, bias=None)

        y_size = out_y.shape
        cropw = y_size[2]-img_size[1]
        croph = y_size[3]-img_size[2]
        out_y = out_y[:,:,cropw//2:cropw//2+img_size[1],croph//2:croph//2+img_size[2]]

        if step < 500:
            total_loss = mse(out_y, y)
        else:
            total_loss = 1 - ssim(out_y, y) #+ tv_loss(out_y)

        total_loss.backward()
        optimizer.step()

        if (step + 1) % opt.save_frequency == 0:
            # print('Iteration %05d' %(step+1))

            save_path = os.path.join(repo_path, opt.save_path, '%s_%s_x.png' % (imgname, step))
            out_x_np = torch_to_np(out_x)
            out_x_np = out_x_np.squeeze()
            cropw, croph = padw, padh
            out_x_np = out_x_np[cropw//2:cropw//2+img_size[1], croph//2:croph//2+img_size[2]]
            out_x_np = np.uint8(255 * out_x_np)
            out_x_np = cv2.merge([out_x_np, cr, cb])
            out_x_np = cv2.cvtColor(out_x_np, cv2.COLOR_YCrCb2BGR)
            cv2.imwrite(save_path, out_x_np)

            save_path = os.path.join(repo_path, opt.save_path, '%s_k.png' % imgname)
            out_k_np = torch_to_np(out_k_m)
            out_k_np = out_k_np.squeeze()
            out_k_np /= np.max(out_k_np)
            imsave(save_path, out_k_np)

            torch.save(net, os.path.join(repo_path, opt.save_path, "%s_xnet.pth" % imgname))
            torch.save(net_kernel, os.path.join(repo_path, opt.save_path, "%s_knet.pth" % imgname))

/content/drive/My Drive/Colab Notebooks/cs231n/project/SelfDeblur-master/datasets/real/jack.jpg
jack


100%|██████████| 1000/1000 [08:43<00:00,  1.91it/s]
